In [1]:
import sys
sys.path.append(r'C:\Users\damod\OneDrive\Área de Trabalho\Projetos Python\dataprocessing')

Importação das bibliotecas

In [2]:
import dataprocessing.datacolect as dc
import re
import pandas as pd
import numpy as np

# 1. Sobre coleta dos dados na Wikipedia

# 2. Rotina de ***scraping***

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_Xbox_Series_X_and_Series_S_games'

In [4]:
soup_class = dc.Soup(url)
soup = soup_class.get_soup()

In [5]:
scrape = dc.Scrape()

In [6]:
games_table = scrape.find_elements(
    tag='tbody', soup=soup,
)
games_table = games_table[1]

In [7]:
games_data = scrape.find_elements(
    tag='tr', soup=games_table,
)
games_data = games_data[2:]

In [10]:
def scraping_games(games: list, init_info: int=1, last_info: int=3):
    """
    Realiza o scraping das informações dos jogos presentes na tabela da lista da Wikipedia.
    """
    scraped_game = dict()
    scraped_games = list()

    for game in games:
        try:
            name_soup = scrape.find_element(tag='th', soup=game)

            if name_soup == None:
                name_soup = scrape.find_element(tag='td', soup=game)
            name = re.sub(r'\n', '', name_soup.get_text())
            scraped_game['name'] = name
            another_infos = scrape.find_elements(tag='td', soup=game)
            games_info = list()

            for another_info in another_infos[init_info:last_info]:
                info = re.sub(r'\n', '', another_info.get_text())
                games_info.append(info)
            scraped_game['infos'] = games_info
            scraped_games.append(scraped_game.copy())
        except AttributeError as e:
            print(f"Attribute Error: {e}")
    return scraped_games


In [11]:
wiki_xbox_series = pd.DataFrame(scraping_games(games_data))

In [12]:
wiki_xbox_series['developer'] = wiki_xbox_series['infos'].apply(lambda x: x[0])
wiki_xbox_series['publisher'] = wiki_xbox_series['infos'].apply(lambda x: x[1])
wiki_xbox_series = wiki_xbox_series.drop(columns='infos')

In [15]:
wiki_xbox_series.to_csv(r'C:\Users\damod\OneDrive\Área de Trabalho\Projetos Python\gamepass\data\xbox_series_wikipedia.csv', index=False)